In [23]:
import socketserver
import socket
import threading
import time

from struct import unpack

"""
今回サーバ↔︎クライアント間で
やりとりするデータの構造

+-----------------------------+
| ヘッダー                            |
+-----------------------------+
| 内容: データサイズ            |
| サイズ: 4byte(固定)        |
+-----------------------------+
+-----------------------------+
| データ                               |
+-----------------------------+
| 内容: 実際のデータ           |
| サイズ: 上記指定のサイズ |
+-----------------------------+
"""
# ヘッダサイズ (固定長)
HEADER_SIZE = 4

# サーバのアドレスとポート
ADDR = "133.53.174.44"
PORT = 50550
MESSAGE = "<reply>2023/01/20 1116:1000.2023/01/20 1116:2000.ID_mm.rejected</reply>"
SHUTDOWN_WORDS = ('stop', 'kill', 'quit', 'shutdown')

class MyTCPHandler(socketserver.BaseRequestHandler):
    """サーバに届いたリクエストを処理するクラス."""

    def handle(self) -> None:
        """
        クライアントからのリクエストを処理する関数.

        クライアントからのリクエストを受信する毎に実行される
        """
        
        client = self.request
        # データ本体の受信

        _data = client.recv(4096).decode()
        print(_data)
        message = MESSAGE
        # シャットダウンコマンドの受信確認
        if _data.lower() in SHUTDOWN_WORDS:
            print('Server is going down')
            self.server.shutdown()
        elif _data == 'mv':
            for i in range(5):
                time.sleep(1)
                message = str(i)
        # データ送信
        print(message)
        client.sendall(bytes(message, 'utf-8'))

# ソケットのアドレスの再利用
class MyTCPServer(socketserver.ThreadingTCPServer, object):
    def server_bind(self) -> None:
#        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.allow_reuse_address = True
        self.socket.bind(self.server_address)          

def start_server() -> None:
    """サーバ起動."""
    with MyTCPServer((ADDR, PORT), MyTCPHandler) as tcps:
        # サーバとしてクライアントのリクエストを待ち続ける
        tcps.serve_forever()
        
start_server()

KeyboardInterrupt: 

In [36]:
import socket
import threading
import socketserver

class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):

    def handle(self):
        data = str(self.request.recv(1024), 'ascii')
        cur_thread = threading.current_thread()
        response = bytes("{}: {}".format(cur_thread.name, data), 'ascii')
        self.request.sendall(response)
        self.server.shutdown()

class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    def server_bind(self) -> None:
#        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.settimeout(3)
        self.allow_reuse_address = True
        self.socket.bind(self.server_address)   

def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        sock.sendall(bytes(message, 'ascii'))
        response = str(sock.recv(1024), 'ascii')
        print("Received: {}".format(response))

# if __name__ == "__main__":
    # Port 0 means to select an arbitrary unused port
HOST, PORT = "133.53.174.44", 50000

server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)
with server:
    ip, port = server.server_address

        # Start a thread with the server -- that thread will then start one
        # more thread for each request
    server_thread = threading.Thread(target=server.serve_forever)
        # Exit the server thread when the main thread terminates
    server_thread.daemon = True
    server_thread.start()
    print("Server loop running in thread:", server_thread.name)

client(HOST, PORT, "Hello World 1")
 #       client(ip, port, "Hello World 2")
 #      client(ip, port, "Hello World 3")

 #       server.shutdown()

Server loop running in thread: Thread-48 (serve_forever)


Exception in thread Thread-48 (serve_forever):
Traceback (most recent call last):
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner


ConnectionRefusedError: [WinError 10061] 対象のコンピューターによって拒否されたため、接続できませんでした。

    self.run()
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\selectors.py", line 324, in select
    r, w, _ = self._select(self._readers, self._writers, [], timeout)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\selectors.py", line 315, in _select
    r, w, x = select.select(r, w, w, timeout)
OSError: [WinError 10038] ソケット以外のものに対して操作を実行しようとしました。
